In [13]:
# https://github.com/llSourcell/pytorch_in_5_minutes/blob/master/demo.py

import torch
from torch.autograd import Variable
import pandas as pd
from random import randint
from ast import literal_eval
import numpy as np
import torch.nn as nn
# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [5]:
#read in relevant data
trainData = np.loadtxt('data/trainData.txt', dtype=np.float32)
queryData = np.loadtxt('data/queryData.txt', dtype=np.float32)
df =  pd.read_pickle("./data/KNN.pkl")
querySize = len(queryData)

In [39]:
# BATCH_SIZE is batch size; INPUT_D is input dimension; HIDDEN_D is hidden dimension; 
BATCH_SIZE, INPUT_D, HIDDEN_D = 1000, 192, 128
ALPHA = 5
LEARNING_RATE = 1e-2
K = 5

# Create random Tensor for trainable features, and wrap them in Variables.
# requires_grad=True indicates that we want to compute gradients wrt these Variables during the backward pass.
anchors = Variable(torch.randn(HIDDEN_D, INPUT_D).type(torch.FloatTensor), requires_grad=True)
biases = Variable(torch.ones(HIDDEN_D).type(torch.FloatTensor) + 16, requires_grad=True)

In [121]:
import torch
from torch.autograd import Function
from torch.nn.modules.distance import PairwiseDistance


class TripletLoss(Function):
    
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.pdist  = PairwiseDistance(2)
        
    def forward(self, anchor, positive, negative):
        pos_dist   = self.pdist.forward(anchor, positive).pow(2)
        neg_dist   = self.pdist.forward(anchor, negative).pow(2)
        
        
        hinge_dist = torch.clamp(self.margin + pos_dist - neg_dist, min = 0.0)
        loss       = torch.mean(hinge_dist)
        return loss

In [101]:
def generateTripplet(index):
    point = torch.tensor([queryData[index]])
    pos = torch.tensor([trainData[df.iloc[index].KNN[randint(0,K)]]])
    neg = torch.tensor([trainData[df.iloc[index].KNN[randint(K, len(df)-1)]]])
    return point, pos, neg

In [114]:
def forward_pass(query):
    return torch.sign(torch.norm(query - anchors, 2, 1) - biases).reshape(1, -1)

In [ ]:
#training
for epoch in range(100):
    
    #generate batch and compute collective loss for batch
    batch_indicies = np.random.choice(querySize, BATCH_SIZE, replace=False)
    loss = 0
    for index in batch_indicies:
        query, pos, neg = generateTripplet(index)
        queryMapped, posMapped, negMapped = [forward_pass(x) for x in [query, pos, neg]]
        triplet_loss = TripletLoss(ALPHA).forward(queryMapped, posMapped, negMapped)
        loss += triplet_loss
    
    print(epoch, loss.data)

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Variables with requires_grad=True.
    # After this we can call var.grad on variables
    loss.backward()

    # Update weights using gradient descent
    biases.data -= LEARNING_RATE * biases.grad.data
    anchors.data -= LEARNING_RATE * anchors.grad.data

    # Manually zero the gradients 
    biases.grad.data.zero_()
    anchors.grad.data.zero_()

0 tensor(19729.)
1 tensor(13016.0010)
2 tensor(15089.)
3 tensor(15773.0010)
4 tensor(16507.0020)


In [87]:
t0 = torch.tensor([[0.], [0.]])
t1 = torch.tensor([[1.], [1.]])
t2 = torch.tensor([[2.], [2.]])
# t2 = torch.tensor([[2.,2.,2.], [3.,3.,3.]], requires_grad = True)
# t3 = torch.sign(torch.norm(t1 - t2, 2, 1) + biases[0:2])
TripletLoss(0).forward(t1, t2, t1)


tensor([1.0000, 1.0000]) tensor([1.0000e-12, 1.0000e-12]) tensor([1.0000, 1.0000]) tensor(1.0000)


tensor(1.0000)